[Main research paper](https://arxiv.org/pdf/2210.11416)

### Trying to build a small and quick chatbot agent. Which can be used for a small robot brain after quantize the model.

Going to use embedding and vector for RAG. We have decided to use mental health dataset so that our bot can help a person espeically aged persons.

In [ ]:
# Install necessary packages
!pip install tqdm transformers spacy datasets langchain_text_splitters langchain-chroma sentence_transformers
!pip install langchain langchain_community transformers pipeline
# !pip install -U datasets
# %pip install llama-index-embeddings-huggingface
# %pip install llama-index-embeddings-instructor
# !pip install llama-index
!pip install sklearn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import numpy as np
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM, T5ForConditionalGeneration, T5Tokenizer
from sentence_transformers import SentenceTransformer
from google.colab import userdata
import urllib.request
from langchain.schema import SystemMessage, HumanMessage, AIMessage
# from langchain_community.document_loaders import HuggingFaceDatasetLoader
import torch
# import torch.nn.functional as F
from google.colab import drive

drive.mount('/content/drive')  # Mount your Google Drive


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
HUGGING_FACE_API_KEY = userdata.get('HUGGING_FACE_API_KEY')

In [ ]:
model_id = "google/flan-t5-small"

In [ ]:
# Check if CUDA is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using device: {device}")

# Load the T5 tokenizer (device-agnostic)
tokenizer = T5Tokenizer.from_pretrained(model_id, legacy=False)

# Load the T5 model
model = T5ForConditionalGeneration.from_pretrained(model_id)

# Move the model to the chosen device (if applicable)
model.to(device)

# for testing
inputs = tokenizer("A step by step recipe to make bolognese pasta:", return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

Using device: cpu


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Pour a cup of bolognese into a large bowl and add the pasta']


In [ ]:
# Function to generate responses from the model
def generate_response(input_text):
    # Tokenize input text
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=1000, truncation=True).input_ids.to(device)

    # Generate response
    outputs = model.generate(
        input_ids,
        max_new_tokens=500,
        num_beams=4,
        no_repeat_ngram_size=2,
        repetition_penalty=2.0,
        top_p=0.92,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True
    )

    # Decode and return the response
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Generate response based on messages
def chat(messages):
    conversation = " ".join([message.content for message in messages if isinstance(message, (HumanMessage, AIMessage))])
    response_text = generate_response(conversation)
    return AIMessage(content=response_text)

In [ ]:
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
]

# Simulate the conversation
response = chat(messages)
print("AI:", response.content)
# Add latest AI response to messages
messages.append(response)


AI: Thank you.


In [ ]:
# Add a new user prompt
new_user_prompt = HumanMessage(content="Who is the inventor of Artificial Intelligence?")
messages.append(new_user_prompt)

In [ ]:
# Get AI response for the new prompt
new_response = chat(messages)
print("AI:", new_response.content)

# Add the new AI response to messages
messages.append(new_response)

AI: samuel roosevelt


In [ ]:
# now create a new user prompt
prompt = HumanMessage(
    content="What is the difference between study and knowledge?"
)
# add to messages
messages.append(prompt)

# send to LLM
res = chat(messages)
print(res.content)

samuel roosevelt


In [ ]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Can you tell me who was invented radio?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)
print(res.content)

samuel roosevelt


In [ ]:
science_facts = [
    "The Earth is not a perfect sphere! It bulges slightly at the equator (about 21 kilometers) due to the centrifugal force caused by its rotation. Imagine spinning a pizza dough in the air - the outward force stretches the dough, causing a similar effect on Earth's shape.",
    "Light travels incredibly fast, at a speed of about 299,792,458 meters per second in a vacuum. This means it takes light only about 8 minutes to travel from the Sun to Earth! However, light speed can slow down slightly when it passes through different materials.",
    "DNA, or deoxyribonucleic acid, is the blueprint of life! This amazing molecule stores the genetic instructions that determine the traits of all living organisms, from the simplest bacteria to complex humans. Each strand of DNA contains a code made up of four chemical bases, which scientists have nicknamed A, T, C, and G.",
    "Gravity is an invisible force that keeps us grounded! It's the attraction between any two objects with mass. The greater the mass of an object, the stronger its gravitational pull. This force is what keeps us on Earth, shapes the orbits of planets around the Sun, and even influences the formation of galaxies.",
    "The periodic table is a scientist's map of the elements! It organizes all known chemical elements based on their atomic number (the number of protons in their nucleus) and their electronic structure. This arrangement reveals fascinating patterns and helps predict the properties of elements, allowing scientists to understand how elements combine to form new materials and molecules.",
]

source_knowledge = "\n".join(science_facts)

In [ ]:
# genearte augmented response from previous document fed into knowledge base
def generate_augmented_response(query):
  # create a new user prompt
  augmented_prompt = f"""<s>[INST] <<SYS>>Use the following context to answer the user's question. If you don't know the answer, just say that you don't know, don't try to make up an answer. <</SYS>>"
 <s>[INST]

 Context: {source_knowledge}
 Question: {query}

 Only return the helpful answer below and nothing else. Helpful answer:[/INST]"""

  # create a new user prompt
  prompt = HumanMessage(
      content=augmented_prompt
  )
  # add to messages
  messages.append(prompt)

  # send to OpenAI
  res = chat(messages)

  return res.content

In [ ]:
query = "What is DNA?"
res = generate_augmented_response(query)
print(res)

deoxyribonucleic acid


In [ ]:
query = "What is Gravity?"
res = generate_augmented_response(query)
print(res)

an invisible force that keeps us grounded!


In [ ]:
from datasets import load_dataset, load_from_disk
import os, glob

# Local dataset path where the dataset will be saved
local_dataset_path = "/content/drive/My Drive/my_project/wizard_of_wikipedia_dataset/"

# Check if the directory exists, create it if not
if not os.path.exists(local_dataset_path):
  os.makedirs(local_dataset_path)  # Create all necessary subdirectories


def download_and_save_dataset():
    try:
        # Download the dataset if not found locally
        dataset = load_dataset("chujiezheng/wizard_of_wikipedia", split="train", cache_dir=local_dataset_path)
        # Save the downloaded dataset for future use
        dataset.save_to_disk(local_dataset_path)
        print("Dataset downloaded and saved successfully.")
        return dataset
    except Exception as e:
        print(f"Failed to download and save the dataset: {e}")
        return None


def is_dataset_downloaded(dataset_path):
  """Checks if the dataset is downloaded by verifying expected files."""
  required_files = ["dataset_info.json"]  # List of essential files

  # Check for presence of split folders (common structure)
  if not os.path.isdir(os.path.join(dataset_path, "train")):  # Check for train split
    return False

  # Check for presence of required files within the train split folder
  for filename in required_files:
    filepath = os.path.join(dataset_path, "train", filename)
    if not os.path.exists(filepath):
      return False

  # Optionally check for additional files (e.g., *.arrow files)
  arrow_pattern = os.path.join(dataset_path, "train", "*.arrow")
  if not any(os.path.isfile(f) for f in glob.glob(arrow_pattern)):  # Check for any *.arrow files
    print("Warning: No Arrow files found. Dataset might be incomplete.")

  return True


def load_local_dataset():
  try:
    # Load the dataset from the local cache
    dataset = load_from_disk(local_dataset_path)
    print("Dataset loaded successfully from disk.")
    return dataset
  except Exception as e:
    print(f"Failed to load dataset from disk: {e}")
    return None


# Check if the folder exists
if not os.path.exists(local_dataset_path):
  os.makedirs(local_dataset_path, exist_ok=True)

# Check if the dataset is downloaded within the folder
if not is_dataset_downloaded(local_dataset_path):
  dataset = download_and_save_dataset()
else:
  dataset = load_local_dataset()

# Print the first example if dataset is loaded
if dataset:
  print(dataset[:1])
  print(dataset.column_names)
  print(dataset.shape)
  print(dataset.info)

  # Assuming 'dataset' is your Hugging Face Dataset
  dataset_features = dataset.features

  # Print feature types for each column
  for feature_name, feature in dataset_features.items():
      print(f"{feature_name}: {feature.dtype}")

else:
  print("Dataset could not be loaded.")


Saving the dataset (0/2 shards):   0%|          | 0/18430 [00:00<?, ? examples/s]

Dataset downloaded and saved successfully.
{'post': [['Science fiction', "I'm a huge fan of science fiction myself! ", 'I agree. One of my favorite forms of science fiction is anything related to time travel! I find it fascinating.', "And that's difficult to do when dealing with time travel. I actually haven't seen the latest Harry Potter movies. Guess it's time to check them out!"]], 'knowledge': [[['no_passages_used __knowledge__ no_passages_used', 'Science fiction __knowledge__ Science fiction (often shortened to SF or sci-fi) is a genre of speculative fiction, typically dealing with imaginative concepts such as futuristic science and technology, space travel, time travel, faster than light travel, parallel universes, and extraterrestrial life.', 'Science fiction __knowledge__ Science fiction often explores the potential consequences of scientific and other innovations, and has been called a "literature of ideas".', 'Science fiction __knowledge__ It usually avoids the supernatural, 

In [ ]:
# Import necessary libraries
import chromadb
import os
from tqdm import tqdm
from langchain_community.document_loaders import HuggingFaceDatasetLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer

# Define the desired folder path within your Drive (replace 'my_project/chromadb' with your path)
persist_directory = "/content/drive/My Drive/my_project/chromadb/"

# Check if the directory exists, create it if not
if not os.path.exists(persist_directory):
  os.makedirs(persist_directory)  # Create all necessary subdirectories

# Create the client with the persist directory
client = chromadb.PersistentClient(path=persist_directory)

# Define collection name
collection_name = "chatbot_knowledgebase"

# Specify columns to embed
# columns_to_embed = ['post', 'response', 'topics']
columns_to_embed = ['post', 'response']

collection_config = {
    "name": collection_name,
    "metadata": {"hnsw:space": "cosine"}
}

# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection = client.get_or_create_collection(**collection_config)

In [ ]:
import logging
from tqdm import tqdm
import gc
import string
import re
import os
import torch
import pickle
from datasets import Dataset
from sentence_transformers import SentenceTransformer
import chromadb
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sklearn.decomposition import PCA
import numpy as np

# Set up logging (optional)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Save the model
model_save_path = "/content/drive/My Drive/my_project/sentence_transformer_model/"
embedding_model_id = "sentence-transformers/all-MiniLM-L6-v2"

# Check if the model is already saved in the specified path
if os.path.exists(model_save_path):
    # If the model exists, load it directly
    sentence_model = SentenceTransformer(model_save_path)
else:
    # If the model does not exist, download it and save it to the specified path
    sentence_model = SentenceTransformer(embedding_model_id, trust_remote_code=True)
    sentence_model.save(model_save_path)

response_tokenizer = T5Tokenizer.from_pretrained(model_id)
response_model = T5ForConditionalGeneration.from_pretrained(model_id)
response_model.to(device)

# Assuming `dataset` is already initialized and loaded
dataset_to_execute = dataset.select(range(500)) # total dataset rows 18430
batch_size = 100

class GenerateEmbeddings:
    def __init__(self, model, batch_size, pca):
        self.model = model
        self.batch_size = batch_size
        self.pca = pca

    def preprocess(self, text):
        """Preprocess text by lowercasing and removing punctuation."""
        text = text.lower()
        text = re.sub(r'__knowledge__', ' ', text)  # Remove custom dividers
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
        return ''.join([c for c in text if c not in string.punctuation])

    def embed_documents(self, texts):
        """Embed a list of texts using the provided model."""
        embeddings = []
        with torch.no_grad():
            for i in range(0, len(texts), self.batch_size):
                batch_texts = texts[i:i + self.batch_size]
                batch_embeddings = self.model.encode([self.preprocess(text) for text in batch_texts], show_progress_bar=False)

                # Apply PCA to reduce the dimensionality to 384 if pca is provided
                if self.pca is not None:
                    reduced_embeddings = self.pca.transform(batch_embeddings)
                    batch_embeddings = [embedding.tolist() for embedding in reduced_embeddings]

                embeddings.extend(batch_embeddings)  # Convert ndarray to list and extend

                torch.cuda.empty_cache()
                gc.collect()
        return embeddings

    def embed_query(self, query):
        return self.embed_documents([self.preprocess(query)])[0]

def generate_response(input_text):
    """Generate a response from the model."""
    input_ids = response_tokenizer(input_text, return_tensors="pt", max_length=1000, truncation=True).input_ids.to(device)

    outputs = response_model.generate(
        input_ids,
        max_new_tokens=500,
        num_beams=4,
        no_repeat_ngram_size=2,
        repetition_penalty=2.0,
        top_p=0.92,
        temperature=0.7,
        eos_token_id=response_tokenizer.eos_token_id,
        do_sample=True
    )

    return response_tokenizer.decode(outputs[0], skip_special_tokens=True)

def flatten_data(data):
    """Flatten nested lists of strings into a single string concatenation."""
    return ' '.join(str(item) for item in data)

def split_text_into_chunks(text, max_length=512):
    tokens = text.split()
    return [' '.join(tokens[i:i + max_length]) for i in range(0, len(tokens), max_length)]

def process_document(doc_id, text, embedding_function):
    """Process and embed a single document."""
    try:
        preprocessed_text = embedding_function.preprocess(text)
        chunks = split_text_into_chunks(preprocessed_text)
        chunk_embeddings = embedding_function.embed_documents(chunks)
        responses = [generate_response(chunk) for chunk in chunks]
        return doc_id, chunk_embeddings, chunks, responses
    except Exception as e:
        logger.error(f"An error occurred while processing document {doc_id}: {e}")
        return None

# Initialize PCA with the target number of components
pca = PCA(n_components=384)

# Fit PCA on some initial data (assuming you have some pre-saved embeddings or initial data to fit)
# Here is an example of how to fit the PCA initially
initial_data = sentence_model.encode(science_facts * 1000)  # Generate some sample data
pca.fit(initial_data)

# Create an embedding function instance
embedding_function = GenerateEmbeddings(sentence_model, batch_size, pca)

# Function to process and save batches
def process_and_save_batches(batch_start, batch_end, processed_data_path, embedding_function):
    processed_documents = {}
    document_ids = []
    all_responses = {}
    progress_bar = tqdm(total=batch_end - batch_start, desc="Processing batches ", unit="batch")

    try:
        for idx in range(batch_start, batch_end):
            for column in columns_to_embed:
                if column not in dataset_to_execute.column_names:
                    logger.warning(f"Column '{column}' not found in dataset, skipping.")
                    continue

                flat_text = flatten_data(dataset_to_execute[column][idx])
                doc_id = f"{column}_id{idx}"
                result = process_document(doc_id, flat_text, embedding_function)
                if result:
                    doc_id, chunk_embeddings, chunks, response_list = result
                    document_ids.extend([f"{doc_id}_chunk{i}" for i in range(len(chunks))])
                    processed_documents.update({f"{doc_id}_chunk{i}": chunk for i, chunk in enumerate(chunks)})
                    all_responses.update({f"{doc_id}_chunk{i}": response_list[i] for i in range(len(response_list))})
                    progress_bar.update(1)


        # Embed the documents using the embedding function
        embeddings = embedding_function.embed_documents(list(processed_documents.values()))

        # Add data to ChromaDB with embeddings, responses, and IDs
        collection.add(
            embeddings=embeddings,
            documents=list(processed_documents.values()),
            metadatas=[{"response": all_responses[doc_id]} for doc_id in document_ids],
            ids=document_ids
        )

        # Persist any remaining changes to ChromaDB
        collection.persist()

        # Print the collection object to verify

        # print(collection)

        # Save the processed documents, document IDs, and responses to disk
        with open(processed_data_path, 'wb') as f:
            pickle.dump({
                'processed_documents': processed_documents,
                'document_ids': document_ids,
                'all_responses': all_responses
            }, f)

        print(f"Processed data saved to {processed_data_path}")

    except Exception as e:
        logger.error(f"An error occurred: {e}")

    finally:
        progress_bar.close()
        torch.cuda.empty_cache()
        gc.collect()

# Process dataset in batches
for start in range(0, len(dataset_to_execute), batch_size):
    end = min(start + batch_size, len(dataset_to_execute))
    processed_data_path = f"/content/drive/My Drive/my_project/processed_data_batch_{start}_{end}.pkl"
    process_and_save_batches(start, end, processed_data_path, embedding_function)

# Print statements to debug the subsequent code
print("Processing completed. Checking subsequent code execution...")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Processing batches : 200batch [09:38,  2.27s/batch]WARNING:chromadb.segment.impl.metadata.sqlite:Insert of existing embedding ID: post_id0_chunk0
ERROR:__main__:An error occurred: 'Collection' object has no attribute 'persist'
Processing batches : 200batch [09:52,  2.96s/batch]
Processing batches : 200batch [10:04,  3.93s/batch]ERROR:__main__:An error occurred: 'Collection' object has no attribute 'persist'
Processing batches : 200batch [10:18,  3.09s/batch]
Processing batches : 200batch [09:43,  3.30s/batch]ERROR:__main__:An error occurred: 'Collection' object has no attribute 'persist'
Processing batches : 200batch [09:55,  2.98s/batch]
Processing batches : 200batch [10:01,  2.61s/batch]ERROR:__main__:An error occurred: 'Collection' object has no attribute 'persist'
Processing batches : 200batch [10:15,  3.08s/batch]
Processing batches : 200batch [09:32,  3.47s/batch]

Processing completed. Checking subsequent code execution...


In [ ]:
# Load the existing collection
collection = client.get_collection(collection_name)

# Verify the loaded collection
print("Collection loaded successfully.")


Collection loaded successfully.


In [ ]:
# import pickle

# # Path to load the processed data
# processed_data_path = "/content/drive/My Drive/my_project/processed_data.pkl"

# # Load the processed documents, document IDs, and responses from disk
# with open(processed_data_path, 'rb') as f:
#     saved_data = pickle.load(f)

# processed_documents = saved_data['processed_documents']
# document_ids = saved_data['document_ids']
# all_responses = saved_data['all_responses']

# print("Processed data loaded successfully")


In [ ]:
import logging
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

def generate_augmented_response(
    query: str,
    embedding_function=embedding_function,
    collection=collection,
    model=sentence_model,
    tokenizer=tokenizer,
    device=device
):
    """
    This function generates a response using the T5 model and retrieves
    relevant knowledge from the provided collection.

    Args:
        query (str): The user's query string.
        embedding_function: The function used to embed the query.
        collection: The ChromaDB collection object containing the knowledge base.
        model: The T5 model used for generating responses.
        tokenizer: The tokenizer used with the T5 model.
        device: The device (CPU or GPU) on which the model is running.

    Returns:
        str: The generated response text.
    """
    try:
        # Embed the query using the provided embedding function
        query_embedding = embedding_function.embed_query(query)

        # Ensure query_embedding is a list
        query_embedding = query_embedding.tolist()

        # Perform similarity search in ChromaDB using the query embedding
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=3  # Retrieve top 3 most similar documents
        )

        if not results or not results['documents']:
            logging.warning("No results returned from ChromaDB similarity search")
            # Fallback mechanism - generate response without knowledge augmentation
            inputs = tokenizer(query, return_tensors="pt", max_length=512, truncation=True).to(device)
            response = model.generate(input_ids=inputs.input_ids, max_length=50, num_beams=4, early_stopping=True)
            return tokenizer.decode(response[0], skip_special_tokens=True)

        # Extract relevant knowledge from the retrieved documents (top 3)
        source_knowledge = "\n".join([
            " ".join(doc) if isinstance(doc, list) else doc for doc in results['documents']
        ])

        # Feed the query and retrieved knowledge into an augmented prompt
        augmented_prompt = f"""Using the contexts below, answer the query.

Contexts:
{source_knowledge}

Query: {query}"""

        # Generate response using the model and the augmented prompt
        with torch.no_grad():  # Disable gradient calculation for efficiency
            inputs = tokenizer(augmented_prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
            response = model.generate(input_ids=inputs.input_ids, max_length=50, num_beams=4, early_stopping=True)

            # Decode the generated token IDs
            response_text = tokenizer.decode(response[0], skip_special_tokens=True)

            return response_text
    except Exception as e:
        logging.error(f"Error in generate_augmented_response: {e}")
        return "Error during response generation."


In [ ]:
# Generate response based on messages
def chat(messages):
    conversation = " ".join([message.content for message in messages if isinstance(message, (HumanMessage, AIMessage))])
    response_text = generate_augmented_response(query=conversation)
    return HumanMessage(content=response_text)

In [ ]:
def chat(messages, embedding_function=embedding_function, collection=collection, model=response_model, tokenizer=tokenizer, device=device):
    conversation = " ".join([message.content for message in messages if isinstance(message, (HumanMessage, AIMessage))])

    # Generate embeddings for the conversation
    query_embedding = embedding_function.embed_query(conversation)

    # Convert embeddings to strings if needed (assuming embeddings are numeric)
    query_embedding = [str(embed) for embed in query_embedding]

    # Example: Query ChromaDB for relevant documents based on metadata (e.g., document IDs or tags)
    relevant_documents = collection.query(query_texts=query_embedding, n_results=3)  # Adjust num_results as needed

    # Extract relevant texts from retrieved documents
    relevant_texts = [doc['text'] for doc in relevant_documents if isinstance(doc, dict) and 'text' in doc]

    if len(relevant_texts) == 0:
        return "Sorry, I couldn't find any relevant information."

    # Tokenize and pad relevant texts
    encoded_inputs = tokenizer(relevant_texts, return_tensors="pt", padding=True, truncation=True)

    input_ids = encoded_inputs["input_ids"].to(device)

    # Generate a response using T5 model
    outputs = model.generate(
        input_ids,
        max_length=100,
        num_beams=4,
        no_repeat_ngram_size=2,
        repetition_penalty=2.0,
        top_p=0.92,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True
    )

    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response_text


In [ ]:
# Example usage:
messages = [HumanMessage(content="Is red hair looks good:")]
response = chat(messages)
print(response)

Sorry, I couldn't find any relevant information.


In [60]:
# Example usage:
response = chat("A step by step recipe to make bolognese pasta:")
print(response)

Sorry, I couldn't find any relevant information.


In [ ]:
# Example usage:
response = chat("Is red hair looks good?")
print(response)

In [ ]:
query = "Which cars has been produced for seven generation?"
res = chat(query)
print(res)
print(chat(res))

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration


def generate_response_with_similarity(
    query: str,
    embedding_function=embedding_function,
    collection=collection,
    model=model,
    tokenizer=tokenizer,
    device=device
    ):
    """
    This function generates a response to a query using the T5 model and performs similarity calculations.

    Args:
        query (str): The user's query string.
        embedding_function: The function used to embed the query.
        collection: The ChromaDB collection object containing the knowledge base.
        model: The T5 model used for generating responses.
        tokenizer: The tokenizer used with the T5 model.
        device: The device (CPU or GPU) on which the model is running.

    Returns:
        tuple: A tuple containing the generated response text and similarity score.
    """
    try:
        # Embed the query using the provided embedding function
        query_embedding = embedding_function.embed_query(query)

        # Perform similarity search in ChromaDB using the query embedding
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=1  # Retrieve top 1 most similar document
        )

        if results is None or 'documents' not in results or not results['documents']:
            # If no similar documents are found, return a default response and similarity score
            return "No relevant information found.", 0.0

        # Extract the retrieved document embedding
        document_embedding = results['embeddings'][0]

        # Convert embeddings to tensors and move to device
        query_tensor = torch.tensor(query_embedding).unsqueeze(0).to(device)
        document_tensor = torch.tensor(document_embedding).unsqueeze(0).to(device)

        # Calculate cosine similarity between query and document embeddings
        similarity_score = torch.nn.functional.cosine_similarity(query_tensor, document_tensor).item()

        # Generate response using the model
        response = model.generate(input_ids=tokenizer.encode(query, return_tensors="pt").to(device))

        # Decode the generated token IDs
        decoded_response = tokenizer.decode(response[0], skip_special_tokens=True)

        return decoded_response, similarity_score

    except Exception as e:
        print(f"An error occurred: {e}")
        return "Error during response generation.", 0.0

# Example usage:
response, similarity_score = generate_response_with_similarity(query="Is red hair looks good?",
                                                               embedding_function=embedding_function,
                                                               collection=collection,
                                                               model=model,
                                                               tokenizer=tokenizer,
                                                               device=device)
print("Generated Response:", response)
print("Similarity Score:", similarity_score)


An error occurred: name 'embed' is not defined
Generated Response: Error during response generation.
Similarity Score: 0.0


In [ ]:
# Assuming you have a collection object for data storage
query_text = "What is the capital of France?"
response_data = search_data_by_vector(query_text)

if response_data:
  print("Found data:")
  print(f"Query: {response_data['query']}")
  print(f"Response: {response_data['response']}")
  print(f"Document: {response_data['document']}")
  print(f"Vector: {response_data['vector']}")
  # Access other details from response_data['complete_response'] if needed
else:
  print("No data found for the query.")


NameError: name 'search_data_by_vector' is not defined

In [ ]:
def augment_prompt(query: str):
    """
    This function attempts to augment the prompt using the knowledge base in ChromaDB.

    Args:
        query (str): The query string.

    Returns:
        str: The augmented prompt (or None if no results or errors occur).
    """
    try:
        # Embed the query using the model
        query_embedding = embedding_function.embed_query(query)

        # Perform similarity search in ChromaDB using the query embedding
        results = collection.query(
            query_embeddings=[query_embedding],
            k=3  # Retrieve top 3 most similar documents
        )

        if not results:
            logging.error("No results returned from ChromaDB similarity search")
            return None

        # Extract relevant knowledge from the retrieved documents (top 3)
        source_knowledge = "\n".join([
            doc for doc in results['documents'][0]  # Assuming top result (index 0)
        ])

        # Feed into an augmented prompt
        augmented_prompt = f"""Using the contexts below, answer the query.

Contexts:
{source_knowledge}

Query: {query}"""

        return augmented_prompt
    except Exception as e:
        logging.error(f"Error in augment_prompt: {e}")
        return None

def chat(messages):
    """
    This function processes a conversation and generates a response using the T5 model.

    Args:
        messages (list): A list of conversation messages (HumanMessage or AIMessage).

    Returns:
        AIMessage: The AI's response message containing the generated text.
    """
    try:
        conversation = " ".join([message.content for message in messages if isinstance(message, (HumanMessage, AIMessage))])
        augmented_prompt = augment_prompt(conversation)

        if not augmented_prompt:
            logging.error("Augmented prompt is None")
            return AIMessage(content="Error generating response.")

        # Generate response using the model (assuming T5ForConditionalGeneration)
        with torch.no_grad():  # Disable gradient calculation for efficiency
            inputs = tokenizer.encode(augmented_prompt, return_tensors="pt")
            response = model.generate(input_ids=inputs, max_length=50, num_beams=4, early_stopping=True)

        # Decode the generated token IDs
        response_text = tokenizer.decode(response[0], skip_special_tokens=True)

        return AIMessage(content=response_text)
    except Exception as e:
        logging.error(f"Error in chat: {e}")
        return AIMessage(content="Error during chat processing.")


In [ ]:
# Example usage
query = "Who is the prisoner in Harry Potter?"
prompt_content = augment_prompt(query)

if not prompt_content:
    raise ValueError("augment_prompt returned None")

prompt = HumanMessage(content=prompt_content)
messages = [prompt]

response = chat(messages)
print("AI:", response.content)

# Add the latest AI response to messages
messages.append(response)

print(augment_prompt(query))


In [ ]:
# Assuming you have a HumanMessage class representing user input
user_message = HumanMessage(content="What is the weather like in Paris today?")

# Call the chat function with the user message
response = chat([user_message])

# Print the AI's response
print("AI:", response.content)


In [ ]:
prompt = HumanMessage(
    content="what kind of fiction are the harry potter?"
)

res = chat(messages + [prompt])
print(res.content)

In [ ]:
# pc.delete_index(index_name)